In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
from tqdm.notebook import tqdm
from selenium.webdriver.common.by import By
import pandas as pd
import time
import urllib
import json
import requests

In [11]:
def click_category(driver, class_O, input_data): 
            #driver과 필터값을 주면 해당필터의 목록을 보여주고
            #값을 받아 해당 카테고리로 이동하는 함수
            #해당 값에 x가 있을경우 이미 클릭한것으로 판단
            #이미 클릭되어있다면 x를 눌러 취소
    
    if class_O == "기간":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[1]/dl/dd/ul"""
    elif class_O == "대분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
    elif class_O == "중분류":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""
    elif class_O == "가격":
        xpath = """//*[@id="goodsRankForm"]/div[1]/div[4]/dl/dd/ul"""
    else:
        print("잘못된 click_category함수 사용")
        return 0
    
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.TAG_NAME,"li")


    word = []

    for date in data_list:
        data_a_list = date.find_elements(By.TAG_NAME,"a")
        for data_a in data_a_list:
            word.append(data_a.get_attribute("innerText"))


    
    try:
        if input_data in word:
            num = word.index(input_data)
            a = data_list[num].find_elements(By.TAG_NAME,"a")
            if len(a) >= 2:
                a[1].click()
            else:
                a[0].click()
            
        else:
            raise TypeError 
    except TypeError:
        print("click_category error : (TypeError) 잘못된 inputdate")
    


def get_more_info(product_numbers):
    product_sellCount = []
    product_target = []
    
    c = 0
    # 사용자 정의 User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    for goods_code in product_numbers:
        # URL을 동적으로 생성
        url = f'https://www.musinsa.com/app/product/goodsview_stats/{goods_code}/0?loc=goods_rank&menu=view'

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # HTTP 요청이 성공적인지 확인

            text_data = response.text  # 응답 텍스트 가져오기
            val = eval(text_data.replace("false","False").replace("null","None").replace("true","True"))
            
            
            product_sellCount.append(val["purchase"]["total"])
            try:
                if val["purchase"]["graph_text"]["sex"] == "f":
                    target = val["purchase"]["graph_text"]["age"]+"대 여성"
                else:
                    target = val["purchase"]["graph_text"]["age"]+"대 남성"
            except KeyError:
                target = "no_data"
                
            product_target.append(target)
            
            
            c+=1
        except requests.exceptions.RequestException as e:
            print(f"Error during request for goods code {goods_code}: {e}")
        except KeyError:
            print(val)
    return product_sellCount, product_target


def get_product_data(driver):
    product_link = []
    product_names = []
    product_brand = []
    product_gender = []
    product_saleRate = []
    product_goods = []
    product_review_count = []
    product_member_price = []
    product_nomal_price = []
    product_tags = []
    product_bigCategory = []
    product_smallCategory = []
    product_rate = []
    product_number=[]
    product_middleCategory = []

    xpath = """//*[@id="goodsRankList"]"""
    data_bar = driver.find_element(By.XPATH, xpath)
    data_list = data_bar.find_elements(By.CLASS_NAME,"list_img")


    goods_tmp = driver.find_element(By.ID,"goodsRankList")
    goods_like = goods_tmp.find_elements(By.NAME,"count")

    for like in goods_like:
        product_goods.append(like.get_attribute("innerText"))

    for data in data_list:
        link = (data.find_element(By.TAG_NAME,"a").get_attribute("href"))
        product_link.append(link)
        re = Request(url=link, headers={"User-Agent":generate_user_agent()})
        page = urlopen(re)
        soup = BeautifulSoup(page, "html.parser")
        product_names.append(soup.find(id="notexist_title").string)
        
        script_org = soup.find_all("script")
        script = script_org[-1].get_text()#
        script_low = (script[script.find("product.state = {")+len("product.state = {")-1:script.rfind(";")+1])
        val = eval(script_low[:script_low.rfind("}")+1].replace("false","False").replace("null","None").replace("true","True"))
        try:
            product_saleRate.append(val["differenceColorGoods"][0]["saleRate"])
        except IndexError:
            product_saleRate.append(0)
        except KeyError:
            product_saleRate.append(0)
        
        product_number.append(val["goodsNo"])
        product_rate.append(data_list.index(data))
        product_brand.append(val["brand"])
        product_gender.append(val["sex"])
        product_bigCategory.append(val["category"]["categoryDepth1Title"].replace("\/",","))
        product_middleCategory.append(val["category"]["categoryDepth2Title"].replace("\/",","))
        product_smallCategory.append(val["category"]["categoryDepth3Title"].replace("\/",","))
        product_member_price.append(val["goodsPriceMemberLevels"][0]["price"])
        product_review_count.append(val["goodsReview"]["totalCount"])
        product_nomal_price.append(val["goodsPrice"]["originPrice"])
        product_tags.append(str(val["goodsTags"]))
    

    product_sellCount, product_target = get_more_info(product_number)
    df = pd.DataFrame()
    df["링크"] = product_link
    df["이름"] = product_names
    df["브랜드"] = product_brand
    df["성별"] = product_gender
    df["할인율"] = product_saleRate
    df["대분류"] = product_bigCategory
    df["중분류"] = product_middleCategory
    df["소분류"] = product_smallCategory
    df["좋아요수"] = product_goods
    df["리뷰개수"] = product_review_count
    df["회원가"] = product_member_price
    df["일반가"] = product_nomal_price
    df["태그"] = product_tags
    df["상품번호"] = product_number
    df["누적 구매수"] = product_sellCount
    df["선호집단"] = product_target
    return df




In [3]:
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/home/rds/amr_ws/edm/data",
        "download.prompt_for_download":False}
options.add_experimental_option("prefs",prefs)
url = "https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page=1&viewType=small&priceMin=&priceMax="
driver = webdriver.Chrome(service=Service("../../driver/chromedriver-linux64/chromedriver"),
                        options=options)
driver.get(url)
driver.maximize_window()
#대기



In [12]:
big_xpath = """//*[@id="goodsRankForm"]/div[1]/div[2]/dl/dd/ul"""
mid_xpath = """//*[@id="goodsRankForm"]/div[1]/div[3]/dl/dd/ul"""

data_bar = driver.find_element(By.XPATH, big_xpath)
big_list = data_bar.find_elements(By.TAG_NAME,"a")

#big_texts = ["가방"]  
big_texts = [ "모자", "아우터", "가방", "상의", "바지", "신발"] # 검색 대분류 목록
mid_black_list = ["환절기 코트","겨울 기타 코트", "패딩 베스트",
                   "사파리/헌팅 재킷", "기타 아우터", 
                   "레깅스", "힐/펌프스", "플랫 슈즈", "신발 용품"]
# 중분류 블랙리스트


for big_text in tqdm(big_texts):
    #data_frame = pd.DataFrame()
    WebDriverWait(driver,10)
    click_category(driver,"대분류",big_text)
    WebDriverWait(driver,10)
    data_bar_mid = driver.find_element(By.XPATH, mid_xpath)
    mid_list = data_bar_mid.find_elements(By.TAG_NAME,"a")
    mid_texts = []
    
    for mid in mid_list:
        if mid.get_attribute("innerText") in mid_black_list:
            continue
        else:
            mid_texts.append(mid.get_attribute("innerText"))

    for mid_text in tqdm(mid_texts):
        WebDriverWait(driver,10)
        click_category(driver,"중분류",mid_text)
        WebDriverWait(driver,10)
        data_frame = get_product_data(driver)
        #data_frame = pd.concat([data_frame, get_product_data(driver)], axis=0)
        click_category(driver,"중분류",mid_text)
    click_category(driver,"대분류",big_text)
    data_frame.to_csv(f"../../data/{big_text}_{time.time()}.csv")#중간저장






  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [25]:
driver.close()

In [ ]:
raise KeyError #한번에 실행 방지용 에러

In [ ]:

#################################################
# 주의 데이터 베이스로 df올리는 블럭임##################
#################################################

import mysql.connector
con = mysql.connector.connect(
    host = "database-1.cd2is2gsweff.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "admin",
    password = "kim82458529",
    database = "amr_base"
)
cur = con.cursor(buffered = True)


sql = "insert into yousinsa values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

for i, row in data_frame.iterrows():
    
    cur.execute(sql,tuple(row))
    con.commit()



con.close()



In [ ]:
####################################################
#for test  #########################################
####################################################
data_frame

'f'